<a href="https://colab.research.google.com/github/INT2-Team2/NeuralNetwork/blob/Davide3/INT2_Group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
%%bash
pip install captum
pip install flask_compress
pip install tqdm
conda install freetype=2.10.4

In [3]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/first_visualization_test')

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance
from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.notebook import tqdm
import time
import pandas as pd

In [5]:
if torch.cuda.is_available(): # use gpu if possible
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [6]:
# only need resize so AlexNet works
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                     (0.2470, 0.2435, 0.2616))])

transform_train = transforms.Compose([transforms.ToTensor(), 
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine(0, (0.1, 0.1)),
                                transforms.RandomAdjustSharpness(1.5, 0.9),
                                transforms.RandomAutocontrast(0.9),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                     (0.2470, 0.2435, 0.2616))])

In [8]:
# get training/test data from CIFAR10 dataset
train_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                        train = True, 
                                        transform = transform_train, 
                                        download = True)
test_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                       train = False, 
                                       transform = transform, 
                                       download = True)

HTTPError: ignored

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 128, 3, 1)
    self.conv2 = nn.Conv2d(128, 128, 3, 1, padding=2)
    self.conv3 = nn.Conv2d(128, 128, 3, 1, padding=2)
    self.conv4 = nn.Conv2d(128, 128, 3, 1, padding=1)
    self.conv5 = nn.Conv2d(128, 128, 3, 1, padding=1)
    self.conv6 = nn.Conv2d(128, 128, 3, 1)
    self.conv7 = nn.Conv2d(128, 256, 3, 1)
    self.conv8 = nn.Conv2d(256, 256, 3, 1)

    self.maxPool = nn.MaxPool2d(2, 2)

    self.drop1 = nn.Dropout(0.3)
    self.drop2 = nn.Dropout(0.3)
    self.drop3 = nn.Dropout(0.2)
    self.drop4 = nn.Dropout(0.4)

    self.BN1 = nn.BatchNorm2d(128)
    self.BN2 = nn.BatchNorm2d(128)
    self.BN3 = nn.BatchNorm2d(256)

    self.fc1 = nn.Linear(256, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)
  
  def forward(self, x):
    x = nn.functional.rrelu(self.conv1(x))
    x = self.maxPool(x)
    x = self.drop1(x)
    x = nn.functional.rrelu(self.conv2(x))
    x = self.maxPool(x)
    x = self.drop2(x)

    x = self.BN1(x)

    x = nn.functional.rrelu(self.conv3(x))

    x = nn.functional.rrelu(self.conv4(x))
    x = self.maxPool(x)
    x = self.drop3(x)

    x = nn.functional.rrelu(self.conv5(x))
    x = self.BN2(x)
    x = nn.functional.rrelu(self.conv6(x))
    x = self.drop4(x)
  # x = self.BN3(x)

    x = nn.functional.rrelu(self.conv7(x))
    x = self.BN3(x)
    x = nn.functional.rrelu(self.conv8(x))

    x = x.reshape(x.shape[0], -1)
    #print(x.shape)
    x = nn.functional.rrelu(self.fc1(x))
    x = nn.functional.rrelu(self.fc2(x))
    x = self.fc3(x)
    return x
  

In [ ]:
# Model creation

model_copy = CNN()
model = CNN().to(device)

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    try:
      npimg = img.numpy()
    except:
      npimg = img.cpu().data.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().data.numpy())
    return preds, [nn.functional.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            train_data.classes[preds[idx]],
            probs[idx] * 100.0,
            train_data.classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig


In [ ]:
def test(test_loader, pbar, print_result = True):
  print("Testing")
  with torch.no_grad():
    correct = 0
    samples = 0

    for i, (images, labels) in enumerate(test_loader):
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      _, predictions = outputs.max(1)

      samples += labels.size(0)
      correct += (predictions == labels).sum()
      time.sleep(0.001)
      pbar.update(labels.size(0))
    if print_result:
      print("Test accuracy was",100*float(correct)/float(samples))
      print()
    else:
      pbar.close()
      return 100*float(correct)/float(samples)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size = 2, 
                                           shuffle = True, 
                                           num_workers = 2)
test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = 2, 
                                          shuffle = False, 
                                          num_workers = 2)
def train(model, epochs, batch_size, learning_rate, weightDecay=0.0015, train_for_minutes = 0, test_per_epoch = True, print_result = True, graphs = False):

  train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           num_workers = 2)
  test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = batch_size, 
                                          shuffle = False, 
                                          num_workers = 2)
  
  train_pbar = tqdm(desc = "Trained images: ", total = 50000, leave = False)
  test_pbar = tqdm(desc = "Tested images: ", total = 10000, leave = False)

  ## Training

  # try other loss functions/optimizers
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay= weightDecay)

  if train_for_minutes: # how many minutes to train for (and then finish current epoch)
    epochs = train_for_minutes*1000
  start = time.time()

  for epoch in tqdm(range(epochs), desc= "Epochs: "):
    train_pbar.reset()
    test_pbar.reset()
    print("Training")
    epoch_loss = 0
    previous_loss = 0
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      # forwards
      outputs = model(images)
      loss = criterion(outputs, labels)

      # backwards
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()

      if i == batch_size-1 and graphs:    # every 1000 mini-batches...

          # ...log the running loss
          writer.add_scalar('training loss',
                            epoch_loss-previous_loss / batch_size,
                            epoch * len(train_loader) + i)

          # ...log a Matplotlib Figure showing the model's predictions on a
          # random mini-batch
          writer.add_figure('predictions vs. actuals',
                             plot_classes_preds(model, images, labels),
                             global_step=epoch * len(train_loader) + i)
          previous_loss = epoch_loss
      time.sleep(0.001)
      train_pbar.update(batch_size)
    print("Epoch %i completed:", i)
    print("Loss was", epoch_loss/len(train_loader))
    print()
    if test_per_epoch:
      test(test_loader, test_pbar)
      
    if train_for_minutes and time.time()-start >= train_for_minutes*60:
      break
      

  if not test_per_epoch:
    if print_result:
      test(test_loader, test_pbar)
    else:
      return (test(test_loader, test_pbar, print_result = False), epoch_loss)
  train_pbar.close()
  test_pbar.close()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def baseline_func(input):
  return 0

visualizer = AttributionVisualizer(
    models=[CNN],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes= train_data.classes,
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[transform],
        )
    ],
    dataset=test_data,
)

In [ ]:
visualizer.render()

In [ ]:
visualizer.serve()

In [ ]:
'''
for weight_decay in np.arange(0.0015, 0.005, 0.0005):
  for learning_rate in np.arange(0.0001, 0.01, 0.0005):
    for batch_size_power in range(2, 5, 1):
      for epochs in range(1, 11):
        (accuracy, loss) = train(CNN().to(device), epochs, 2**batch_size_power, learning_rate, weight_decay, 0, False, False, False)
        hparams_dict = {"Epochs" : epochs, "Batch Size": 2**batch_size_power, "Learning Rate" : learning_rate, "Weight Decay" : weight_decay}
        writer.add_hparams(hparams_dict, {"Accuracy":accuracy, "Training Loss": loss})
'''

In [ ]:
train(model, 400, 128, 0.0001)

In [ ]:
train(model, 400, 128, 0.0001)

In [ ]:
classes = torch.tensor([2, 3, 5])
indices = (torch.tensor(train_data.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
cats_and_dogs_data = torch.utils.data.Subset(train_data, indices)
cats_and_dogs_loader = torch.utils.data.DataLoader(cats_and_dogs_data, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = 16, 
                                          shuffle = False, 
                                          num_workers = 2)
  
train_pbar = tqdm(desc = "Trained images: ", total = 15000, leave = False)
test_pbar = tqdm(desc = "Tested images: ", total = 10000, leave = False)

  ## Training

  # try other loss functions/optimizers
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.0001, weight_decay= 0.0015)

for epoch in tqdm(range(2), desc= "Epochs: "):
    train_pbar.reset()
    test_pbar.reset()
    print("Training")
    epoch_loss = 0
    previous_loss = 0
    for i, (images, labels) in enumerate(cats_and_dogs_loader):
      images = images.to(device)
      labels = labels.to(device)

      # forwards
      outputs = model(images)
      loss = criterion(outputs, labels)

      # backwards
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()

      time.sleep(0.001)
      train_pbar.update(16)
    print("Loss was", epoch_loss/len(train_loader))
    print()
    test(test_loader, test_pbar)
train_pbar.close()
test_pbar.close()

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_graph(model_copy, images)
writer.add_image('first_visualization_test', img_grid)
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/first_visualization_test/

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size = 16, 
                                           shuffle = True, 
                                           num_workers = 2)
test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = 16, 
                                          shuffle = False, 
                                          num_workers = 2)
  
train_pbar = tqdm(desc = "Trained images: ", total = 50000, leave = False)
test_pbar = tqdm(desc = "Tested images: ", total = 10000, leave = False)

  ## Training

  # try other loss functions/optimizers
criterion = nn.CrossEntropyLoss(weight = torch.add(1, nn.functional.normalize((confusion_matrix.diag()/confusion_matrix.sum(1)).neg(),p=1.0, dim = 0).to(device)))
optimizer = torch.optim.Adam(model.parameters(), 0.0001, weight_decay= 0.0015)


for epoch in tqdm(range(10), desc= "Epochs: "):
  confusion_matrix = torch.zeros(10, 10)
  with torch.no_grad():
      for i, (inputs, classes) in enumerate(test_loader):
          inputs = inputs.to(device)
          classes = classes.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          for t, p in zip(classes.view(-1), preds.view(-1)):
                  confusion_matrix[t.long(), p.long()] += 1
  criterion.weight = torch.add(1, nn.functional.normalize((confusion_matrix.diag()/confusion_matrix.sum(1)).neg(),p=1.0, dim = 0).to(device))
  train_pbar.reset()
  test_pbar.reset()
  print("Training")
  epoch_loss = 0
  previous_loss = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forwards
    outputs = model(images)
    loss = criterion(outputs, labels)

      # backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

    time.sleep(0.001)
    train_pbar.update(16)
  print("Loss was", epoch_loss/len(train_loader))
  print()
  test(test_loader, test_pbar)
      
train_pbar.close()
test_pbar.close()

In [ ]:
confusion_matrix = torch.zeros(10, 10)
with torch.no_grad():
   for i, (inputs, classes) in enumerate(test_loader):
          inputs = inputs.to(device)
          classes = classes.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          for t, p in zip(classes.view(-1), preds.view(-1)):
                  confusion_matrix[t.long(), p.long()] += 1
test_accuracy_per_class = pd.DataFrame(confusion_matrix.diag()/confusion_matrix.sum(1), test_data.classes, ["Accuracy"])
plt.figure(figsize=(20, 10))
plt.grid(color='black', axis='y', linestyle='dashed', zorder = 1)
bar = plt.bar(test_accuracy_per_class.index, height=test_accuracy_per_class.Accuracy*100, zorder = 2)
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.2f}%', ha='center', va='bottom', zorder = 3)
plt.yticks(range(0, 101, 5))
plt.axhline((confusion_matrix.diag()/confusion_matrix.sum(1)).mean()*100, color='r', linestyle='--')
plt.show()

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(confusion_matrix, xticklabels=test_data.classes, yticklabels=test_data.classes, square=True, annot=True, cmap='Blues', cbar=False)